In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import torch
import matplotlib.pyplot as plt
import seaborn.objects as so

from opf.test import load_run, test_run

mpl.rcParams["text.usetex"] = False

sns.set_style("white")
sns.set_context("paper")

torch.set_float32_matmul_precision("high")

In [ ]:
import wandb

api = wandb.Api()
runs = api.runs(
    "damowerko-academic/opf",
    filters={
        "$and": [{"config.max_epochs": 5000}, {"created_at": {"$gt": "2025-03-11"}}]
    },
)

In [ ]:
run_dict = {}
for run in runs:
    case_name = run.config["case_name"]
    # Rename case names to pretty ones
    if "case30_ieee" in case_name:
        case_name = "IEEE 30"
    elif "case57_ieee" in case_name:
        case_name = "IEEE 57"
    elif "case118_ieee" in case_name:
        case_name = "IEEE 118"
    elif "case179_goc" in case_name:
        case_name = "GOC 179"
    elif "case200_activ" in case_name:
        case_name = "ACTIV 200"
    elif "case300_ieee" in case_name:
        case_name = "IEEE 300"
    elif "case1354_pegase" in case_name:
        case_name = "PEGASE 1354"

    tags = set(run.tags)
    if {"supervised"} == tags:
        model_name = "Supervised"
    elif {"supervised", "augmented"} == tags:
        model_name = "Supervised + Augmented"
    elif {"pointwise"} == tags:
        model_name = "Pointwise"
    elif {"pointwise", "supervised-warmup"} == tags:
        model_name = "Pointwise + Supervised Warmup"
    elif {"shared"} == tags:
        model_name = "Shared"
    elif {"shared", "supervised-warmup"} == tags:
        model_name = "Shared + Supervised Warmup"
    elif {"hybrid"} == tags:
        model_name = "Hybrid"
    elif {"hybrid", "supervised-warmup"} == tags:
        model_name = "Hybrid + Supervised Warmup"

    tags = run.tags
    run_id = run.id
    if case_name not in run_dict:
        run_dict[case_name] = {}
    run_dict[case_name][model_name] = run_id

case_names = [
    "IEEE 30",
    "IEEE 57",
    "IEEE 118",
    "GOC 179",
    "ACTIV 200",
    "IEEE 300",
    # "PEGASE 1354",
]
model_names = [
    "Supervised",
    "Supervised + Augmented",
    "Pointwise",
    "Pointwise + Supervised Warmup",
    "Shared",
    "Shared + Supervised Warmup",
    "Hybrid",
    "Hybrid + Supervised Warmup",
]
model_type_dict = {
    "Supervised": "Supervised",
    "Supervised + Augmented": "Supervised",
    "Pointwise": "Dual-Pointwise",
    "Pointwise + Supervised Warmup": "Dual-Pointwise",
    "Shared": "Dual-Shared",
    "Shared + Supervised Warmup": "Dual-Shared",
    "Hybrid": "Dual-Pointwise",
    "Hybrid + Supervised Warmup": "Dual-Pointwise",
}

In [ ]:
dfs = []
for case_name in run_dict:
    if case_name != "GOC 179":
        continue
    for model_name, id in run_dict[case_name].items():
        if id == "":
            continue
        df = test_run(id, load_existing=False, project=True, clamp=False)
        df = df.assign(id=id, case_name=case_name, model_name=model_name)
        dfs.append(df)

df = pd.concat(dfs)

In [ ]:
index = pd.MultiIndex.from_product([case_names, model_names], names=["Case", "Model"])
df["optimality_gap"] = df["test/cost"] / df["acopf/cost"] - 1
df_summary = (
    df.groupby(["case_name", "model_name"])[
        [
            "optimality_gap",
            "test/inequality/error_mean",
            "test/inequality/error_max",
            "test/equality/error_mean",
            "test/equality/error_max",
        ]
    ]
    .mean()
    .reindex(index)
)
df_summary.query(
    "Model in ['Supervised + Augmented', 'Pointwise', 'Shared']"
).style.format("{:.4f}", na_rep="--").set_table_styles(
    [
        {
            "selector": f"tr:nth-child({i * 8})",
            "props": [("border-bottom", "2px solid black")],
        }
        for i in range(1, 7)
    ]
)

In [ ]:
# average performance across all cases
df_summary_average = df_summary.groupby("Model").mean()
df_summary_average.style.format("{:.2%}", na_rep="--")

In [ ]:
(
    so.Plot(
        df_summary_average[
            [
                "optimality_gap",
                "test/inequality/error_mean",
                "test/inequality/error_max",
            ]
        ]
        .melt(ignore_index=False)
        .reset_index()
        .assign(model_type=lambda df: df["Model"].map(model_type_dict)),
        x="value",
        y="Model",
        color="model_type",
    )
    .add(so.Bar())
    .facet(col="variable")
    .scale(x=so.Continuous().label(like="{x:.2%}").tick(count=5))
    .limit(x=(0, 0.2))
    .layout(size=(16, 8))
)

In [ ]:
(
    so.Plot(
        df_summary[
            [
                "optimality_gap",
                "test/inequality/error_mean",
                "test/inequality/error_max",
            ]
        ]
        .melt(ignore_index=False)
        .reset_index()
        # .query("not Model.str.contains('Shared') and not Model.str.contains('Hybrid')")
        .assign(model_type=lambda df: df["Model"].map(model_type_dict)),
        x="value",
        y="Model",
        color="model_type",
    )
    .facet(row="Case", col="variable")
    .add(so.Bar())
    .share(x=False)
    .scale(x=so.Continuous().label(like="{x:0.2%}").tick(count=5))
    .layout(size=(16, 16))
    .plot()
)

In [ ]:
import re

# column index
constaint_type = [
    "active_power",
    "reactive_power",
    "voltage_magnitude",
    "forward_rate",
    "backward_rate",
    "bus_active_power",
    "bus_reactive_power",
    "bus_reference",
]
statustic = ["mean", "max"]
column_index = pd.MultiIndex.from_product(
    [constaint_type, statustic], names=["Constraint", "Statistic"]
)

constraint_pattern = re.compile(
    r"test/(equality|inequality)/([^/]+)/(error_mean|error_max)"
)
metric_columns = {}
for column in df.columns:
    match = constraint_pattern.match(column)
    if not match:
        continue
    _, constraint_name, error_type = match.groups()
    error_type = error_type.replace("error_", "")
    metric_columns[column] = (constraint_name, error_type)

id_vars = ["case_name", "model_name"]
df_melted = df[id_vars + list(metric_columns.keys())].melt(
    id_vars=id_vars,
    value_vars=list(metric_columns.keys()),
)
df_melted["constraint_type"] = df_melted["variable"].apply(
    lambda x: metric_columns[x][0]
)
df_melted["statistic"] = df_melted["variable"].apply(lambda x: metric_columns[x][1])
df_melted = df_melted.drop(columns=["variable"])

df_pivoted = (
    df_melted.groupby(id_vars + ["constraint_type", "statistic"])
    .mean()
    .pivot_table(
        index=id_vars,
        columns=["constraint_type", "statistic"],
        values="value",
    )
    .reindex(index, columns=column_index)
)
df_pivoted.style.format("{:.4f}", na_rep="--").set_table_styles(
    [
        {
            "selector": f"tr:nth-child({i * 8})",
            "props": [("border-bottom", "2px solid black")],
        }
        for i in range(1, 6)
    ]
)